In [52]:
import os

In [53]:
%pwd

'D:\\office\\vc'

In [54]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [55]:
from cloner.constants import *
from cloner.utils.common import read_yaml, create_directories

In [77]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):
            
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])  #using box return type we can call the key-value pair with ".", here we are creating directories mentioned in artifacts

    def get_data_ingestion_config(self)->DataIngestionConfig:
          config=self.config.data_ingestion #storing configuration  inside config variable
          create_directories([config.root_dir]) #creating artifact directory

          data_ingestion_config=DataIngestionConfig( #defining return type
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          ) 

          return data_ingestion_config 

In [ ]:
import os
import urllib.request as request
import zipfile
from cloner import logger
from cloner.utils.common import get_size

In [1]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file): #can use gdown api for google drive downloads for future
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

NameError: name 'DataIngestionConfig' is not defined

In [76]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-19 17:47:24,875: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-19 17:47:24,882: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-19 17:47:24,886: INFO: common: created directory at: artifacts]
[2025-05-19 17:47:24,890: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-19 17:47:24,894: INFO: 1734585181: File already exists of size: ~ 107687 KB]
